# Projeto 1 - Ciência dos Dados

Nome: Enricco Gemha

Nome: Marcelo Rabello Barranco

Nome: Rafael Coca Leventhal

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import nltk

nltk.download('rslp');
nltk.download('stopwords');

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\rafae\OneDrive - Insper - Institudo de Ensino e Pesquisa\2º sem\codes\projeto-codes\projeto1-cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [44]:
filename = 'iphone.xlsx'

In [45]:
train = pd.read_excel(filename)
train

,Treinamento,RELEVÂNCIA
0,- pego meu iphone 15 pro max 6g\n- ligo pra mi...,0
1,@venusbillie ai fica tao mais bonito no iphone...,4
2,mas tambem quem compra iphone 6 em 2021 tem ma...,4
3,"feliz pra crl , pegar meu iphone agr 😅",4
4,@tatielly_keys vc tem iphone?,0
...,...,...
745,minha mãe quer comprar o iphone 12 pra ela e n...,4
746,nunca precisei mexer no meu iphone se antigo p...,0
747,era só um carregador de iphone 😪,0
748,"tô doida com essa película privacidadeee, vi o...",3


In [46]:
test = pd.read_excel(filename, sheet_name='Teste')
test.head(5)

,Teste,RELEVÂNCIA
0,minha mãe podia me dá um iphone só de pena no ...,4
1,qria fazer a trend da tv alem de ser pequena n...,4
2,papo de q o whatsapp de quem tem iphone vai pa...,3
3,iphone 13 vai esperar mais um pouco pq a gata ...,4
4,alguém sabe se quando quebra a tela do iphone ...,3


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Produto: iPhone (dispensa descrições)

* MUITO IRRELEVANTE: falando de outro assunto, não envolve o iPhone, pessoas que não sabem tweetar ou se comunicar (Ex: só colocou uma hashtag)
* IRRELEVANTE: anúncios de venda (EX: venha comprar no magalu)
* NEUTRO: piada sobre o iPhone (EX: iPhone é o Corsa em miniatura kkkkkk)
* RELEVANTE: comentário indireto relacionado ao iPhone (EX: meu professor de ciências passou 30 minutos só falando do novo iPhone dele)
* MUITO RELEVANTE: falando objetivamente do iPhone, tipo opinião, dúvida ou desejo de comprar (EX: iPhone 13 vai esperar um pouco para chegar em minhas mãos)

- MUITO IRRELEVANTE = 0
- IRRELEVANTE = 1
- NEUTRO = 2
- RELEVANTE = 3
- MUITO RELEVANTE = 4

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [47]:
def clean_data(text):
    # removendo sinais de pontuação
    text = re.sub(r'[!-.:?;[\]/]', '', text)

    # tokenizing
    tokenizer = nltk.tokenize.casual.TweetTokenizer()
    text = tokenizer.tokenize(text)

    # stemming
    stemmer = nltk.stem.RSLPStemmer()
    text = map(stemmer.stem, text)

    # removendo stop words
    stop_words = nltk.corpus.stopwords.words('portuguese')
    text = [w for w in text if w not in stop_words]

    return text


In [48]:
# preparando os dados
train.loc[:, 'Treinamento'] = train.Treinamento.apply(clean_data)
train.loc[:, 'RELEVÂNCIA'] = train.RELEVÂNCIA.map({0: 'muito irrelevante',
                                                   1: 'irrelevante',
                                                   2: 'neutro',
                                                   3: 'relevante',
                                                   4: 'muito relevante'})
train

,Treinamento,RELEVÂNCIA
0,"[peg, iphon, 15, pro, max, 6g, lig, pra, minh,...",muito irrelevante
1,"[@venusbilli, ai, fic, tao, bonit, iphon, q, d...",muito relevante
2,"[tamb, qu, compr, iphon, 6, 2021, fud, mesm, k...",muito relevante
3,"[feliz, pra, crl, peg, iphon, agr, 😅]",muito relevante
4,"[@tatielly_key, vc, iphon]",muito irrelevante
...,...,...
745,"[minh, mãe, qu, compr, iphon, 12, pra, qu, tro...",muito relevante
746,"[nunc, precis, mex, iphon, antig, pra, nad, ag...",muito irrelevante
747,"[carreg, iphon, 😪]",muito irrelevante
748,"[tô, doid, ess, películ, privacidadee, vi, sto...",relevante


In [49]:
train.RELEVÂNCIA.describe()

count                   750
unique                    5
top       muito irrelevante
freq                    208
Name: RELEVÂNCIA, dtype: object

In [50]:
# construindo classificador
categorias = ['muito irrelevante', 'irrelevante',
              'neutro', 'relevante', 'muito relevante']

# número de palavras/emojis únicos presentes em toda a base de dados
palavras_unicas = set(sum(train.Treinamento, []))

# lista de palavras por categoria
palavras_por_categoria = {
    categoria: sum(train[train.RELEVÂNCIA == categoria].Treinamento, [])
    for categoria in categorias
}

# número de ocorrências de palavra por categoria
ocorrencia_de_palavras_por_categoria =  {
    categoria: {
        palavra: palavras_por_categoria[categoria].count(palavra)
        for palavra in palavras_unicas
    }
    for categoria in categorias
}


def prob_frase(categoria, frase):
    '''
    Calcula a probabilidade de frase estar em categoria
    '''
    # limpeza de frase caso forneciada no formato de str
    if frase is str:
        frase = clean_data(frase)
    
    # cálculo da probabilidade
    return np.array(list(
        # probabilidade de cada palavra com suavização de laplace
        ((ocorrencia_de_palavras_por_categoria[categoria][palavra] + 1
        if palavra in palavras_unicas else 1) /
         (len(palavras_por_categoria[categoria]) + len(palavras_unicas)))
        for palavra in frase
    )).prod()  # produto da probabilidade de cada palavra


def classificador(frase):
    '''
    Retorna a categoria com a maior probabilidade de conter frase
    '''
    return max(
        categorias, key=lambda categoria: prob_frase(categoria, frase)
    )

In [51]:
test

,Teste,RELEVÂNCIA
0,minha mãe podia me dá um iphone só de pena no ...,4
1,qria fazer a trend da tv alem de ser pequena n...,4
2,papo de q o whatsapp de quem tem iphone vai pa...,3
3,iphone 13 vai esperar mais um pouco pq a gata ...,4
4,alguém sabe se quando quebra a tela do iphone ...,3
...,...,...
245,qual iphone compensa pegar agora em 2021?,3
246,meta de maria:\nganhar um iphone 12 da melhor ...,4
247,"usando o combo iphone, macbook e airpods é per...",3
248,medo. prefiro ficar sem iphone https://t.co/ro...,4


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [52]:
test.loc[:, 'Teste'] = test.Teste.apply(clean_data)
test.loc[:, 'RELEVÂNCIA'] = test.RELEVÂNCIA.map({0: 'muito irrelevante',
                                                   1: 'irrelevante',
                                                   2: 'neutro',
                                                   3: 'relevante',
                                                   4: 'muito relevante'})
test

,Teste,RELEVÂNCIA
0,"[minh, mãe, pod, dá, iphon, pen, aniversári]",muito relevante
1,"[qri, faz, trend, tv, alem, ser, pequen, n, te...",muito relevante
2,"[pap, q, whatsapp, qu, iphon, vai, par, alguém...",relevante
3,"[iphon, 13, vai, esper, pouc, pq, gat, troc, p...",muito relevante
4,"[alguém, sab, qu, quebr, tel, iphon, val, pen,...",relevante
...,...,...
245,"[iphon, compens, peg, agor, 2021]",relevante
246,"[met, mar, ganh, iphon, 12, melhor, amig]",muito relevante
247,"[us, comb, iphon, macbook, airpod, perfeit, di...",relevante
248,"[med, prefir, fic, iphon, httpstcoroompbrncj]",muito relevante


In [53]:
classificador(['med', 'prefir', 'fic', 'iphon', 'httpstcoroompbrncj'])

'neutro'

In [54]:
test['Acertos'] = test.Teste.apply(classificador) == test.RELEVÂNCIA
pd.crosstab(test.Acertos, test.RELEVÂNCIA, normalize='columns')

RELEVÂNCIA,irrelevante,muito irrelevante,muito relevante,neutro,relevante
Acertos,,,,,
False,0.894737,0.627907,0.5,0.616667,0.890625
True,0.105263,0.372093,0.5,0.383333,0.109375


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**